In [ ]:
import math
import numpy as np
from collections import defaultdict

# Training Data
train_sentences = [
    "The_DET cat_NOUN sleeps_VERB",
    "A_DET dog_NOUN barks_VERB",
    "The_DET dog_NOUN sleeps_VERB",
    "My_DET dog_NOUN runs_VERB fast_ADV",
    "A_DET cat_NOUN meows_VERB loudly_ADV",
    "Your_DET cat_NOUN runs_VERB",
    "The_DET bird_NOUN sings_VERB sweetly_ADV",
    "A_DET bird_NOUN chirps_VERB"
]

tagged_data = []
for sentence in train_sentences:
    word_tag_pairs = sentence.strip().split()
    tagged_sentence = [tuple(wt.rsplit('_', 1)) for wt in word_tag_pairs]
    tagged_data.append(tagged_sentence)

class HMM:
    def __init__(self):
        self.states = set()
        self.vocab = set()
        self.start_probs = defaultdict(float)
        self.trans_probs = defaultdict(lambda: defaultdict(float))
        self.emit_probs = defaultdict(lambda: defaultdict(float))

    def train(self, tagged_sentences):
        tag_counts = defaultdict(int)
        transition_counts = defaultdict(lambda: defaultdict(int))
        emission_counts = defaultdict(lambda: defaultdict(int))
        start_counts = defaultdict(int)

        for sentence in tagged_sentences:
            prev_tag = None
            for i, (word, tag) in enumerate(sentence):
                self.states.add(tag)
                self.vocab.add(word.lower())
                tag_counts[tag] += 1
                emission_counts[tag][word.lower()] += 1

                if i == 0:
                    start_counts[tag] += 1
                if prev_tag is not None:
                    transition_counts[prev_tag][tag] += 1
                prev_tag = tag

        total_starts = sum(start_counts.values())
        for tag in self.states:
            self.start_probs[tag] = start_counts[tag] / total_starts
            for next_tag in self.states:
                self.trans_probs[tag][next_tag] = (
                    transition_counts[tag][next_tag] / tag_counts[tag]
                )
            for word in self.vocab:
                self.emit_probs[tag][word] = (
                    emission_counts[tag][word] / tag_counts[tag]
                )

    def viterbi(self, sentence):
        V = [{}]
        path = {}

        for tag in self.states:
            V[0][tag] = self.start_probs[tag] * self.emit_probs[tag].get(sentence[0], 1e-6)
            path[tag] = [tag]

        for t in range(1, len(sentence)):
            V.append({})
            new_path = {}

            for curr_tag in self.states:
                max_prob, best_prev_tag = max(
                    (V[t - 1][prev_tag] * self.trans_probs[prev_tag].get(curr_tag, 1e-6) *
                     self.emit_probs[curr_tag].get(sentence[t], 1e-6), prev_tag)
                    for prev_tag in self.states
                )
                V[t][curr_tag] = max_prob
                new_path[curr_tag] = path[best_prev_tag] + [curr_tag]

            path = new_path

        max_final_prob, best_final_tag = max((V[-1][tag], tag) for tag in self.states)
        return path[best_final_tag], max_final_prob

model = HMM()
model.train(tagged_data)

# Test Sentences
test_sentences = [
    "The cat meows",
    "My dog barks loudly"
]

print(" Viterbi Tagging Results \n")
for sentence in test_sentences:
    tokens = [w.lower() for w in sentence.split()]
    best_tags, prob = model.viterbi(tokens)
    print(f"Sentence: {sentence}")
    print("Tags   :", " --> ".join(best_tags))
    print(f"Probability: {prob:.6f}\n")


 Viterbi Tagging Results 

Sentence: The cat meows
Tags   : DET --> NOUN --> VERB
Probability: 0.017578

Sentence: My dog barks loudly
Tags   : DET --> NOUN --> VERB --> ADV
Probability: 0.000732

